In [ ]:
# Papermill params
ratio = 0.9          # Train-Test split ratio
attempts = 20        # Number of times to run
width = 256
depth = 5
learning_rate = 5e-2
dropout = 0.0
regularization = 1e-8
epsilon = 1e-7

# Neural network

In this notebook we set up the neural networks with VAMPNet scoring functions and train them for different output sizes and estimate errors by bootstrap aggregation. This notebook can be used with `papermill` to run all cells automatically with given parameters. We first define the imports and useful utility functions.

In [ ]:
%run model.py

## Data
### Trajectories
Trajectories were acquired in five rounds of 1024 simulations each, totalling 5119 runs (one simulation failed to run) at 278 K in the $NVT$ ensemble. Postprocessing involved removing water, subsampling to 250 ps timesteps, and making molecules whole.

In [ ]:
sim_names = ("red", "ox")
top, trajs = {}, {}
trajs = {k: sorted(glob("trajectories/{0}/r?/traj*.xtc".format(k))) for k in sim_names}
top = {k: "trajectories/{0}/topol.gro".format(k) for k in sim_names}
KBT = 2.311420 # 278 K
nres = 42
traj_rounds = {
    "red": [1024, 1023, 1024, 1024, 1024],
    "ox": [1024, 1024, 1023],
}

# This is only really necessary for the residues in the plots
topo = md.load_topology(top["red"])

We use minimum distances as features for the neural network:

In [ ]:
inpcon = {}
for k in sim_names:
    feat = pe.coordinates.featurizer(top[k])
    feat.add_residue_mindist()
    inpcon[k] = pe.coordinates.source(trajs[k], feat)

In [ ]:
lengths, nframes = {}, {}
for i, k in enumerate(sim_names):
    # Switch for full version:
    # lengths[k] = sort_lengths(inpcon[k].trajectory_lengths(), traj_rounds[k])
    lengths[k] = [inpcon[k].trajectory_lengths()]
    nframes[k] = inpcon[k].trajectory_lengths().sum()

In [ ]:
print("\t\t" + "\t\t".join(sim_names))
print("\n".join((
    "Trajs: \t\t" + "\t\t".join("{0}".format(len(trajs[k])) for k in sim_names),
    "Frames: \t" + "\t\t".join("{0}".format(nframes[k]) for k in sim_names),
    "Time: \t\t" + "\t".join("{0:5.3f} µs".format(inpcon[k].trajectory_lengths().sum() * 0.00025)
                           for k in sim_names)
)))

## VAMPNet
VAMPNet[1] is composed of two lobes, one reading the system features $\mathbf{x}$ at a timepoint $t$ and the other after some lag time $\tau$. In this case the network reads all minimum inter-residue distances (780 values) and sends them through 5 layers with 256 nodes each. The final layer uses between 2 and 8 *softmax* outputs to yield a state assignment vector $\chi: \mathbb{R}^m \to \Delta^{n}$ where $\Delta^{n} = \{ s \in \mathbb{R}^n \mid 0 \le s_i \le 1, \sum_i^n s_i = 1 \}$ representing the probability of a state assignment. One lobe thus transforms a system state into a state occupation probability. We can also view this value as a kind of reverse ambiguity, i.e. how sure the network is that the system is part of a certain cluster. These outputs are then used as the input for the VAMP scoring function. We use the new enhanced version with physical constraints[2], particularly the ones for positive entries and reversibility.

[1] Mardt, A., Pasquali, L., Wu, H. & Noé, F. VAMPnets for deep learning of molecular kinetics. Nat Comms 1–11 (2017). doi:10.1038/s41467-017-02388-1

[2] Mardt, A., Pasquali, L., Noé, F. & Wu, H. Deep learning Markov and Koopman models with physical constraints. arXiv:1912.07392 [physics] (2019).

### Data preparation
We use minimum residue distances as input ($\frac{N(N-1)}{2}$ values, where $N$ is the number of residues) and first normalize the data:

In [ ]:
for k in sim_names:
    filename = "intermediate/mindist-780-{0}.npy".format(k)
    if not os.path.exists(filename):
        print("No mindist file for {0} ensemble, calculating from scratch...".format(k))
        con = np.vstack(inpcon[k].get_output())
        np.save(filename, con)

In [ ]:
input_flat, input_data = {}, {}
for k in sim_names:
    raw = np.load("intermediate/mindist-780-{0}.npy".format(k))
    raw_mean, raw_std = raw.mean(axis=0), raw.std(axis=0)
    input_flat[k] = (raw - raw_mean) / raw_std
    input_data[k] = [(r - raw_mean) / raw_std for r in unflatten(raw, lengths[k])]

### Neural network hyperparameters
To allow for a larger hyperparameter search space, we use the self-normalizing neural network approach by Klambauer *et al.* [2], thus using SELU units, `AlphaDropout` and normalized `LeCun` weight initialization. The other hyperparameters are defined at the beginning of this notebook.

[2] Klambauer, G., Unterthiner, T., Mayr, A. & Hochreiter, S. Self-Normalizing Neural Networks. arXiv.org cs.LG, (2017).

In [ ]:
lag = 20                            # Lag time
n_dims = input_data[k][0].shape[1]  # Input dimension
nres = 42                           # Number of residues
dt = 0.25                           # Trajectory timestep in ns
bs_frames = 1000000                 # Number of frames in the bootstrap sample

# The oxidised model was trained without batch normalisation
batchnorm = {"red": True, "ox": False}

# Comment for full version:
bs_frames = nframes
attempts = 2
outsizes = np.array([4])

### Convergence
We would ideally like to see how converged our ensemble is with respect to the timescales and stationary distribution given by our model. We thus build trial models with different numbers of trajectories:

In [ ]:
n = 4
for k in ("red", "ox"):
    filename = "intermediate/temp-k-conv-{0}-{1}.npy".format(k, n)
    k_conv = np.empty((len(traj_rounds[k]), attempts, n, n))
    for j, nt in enumerate(np.cumsum(traj_rounds[k])):
        for i in range(attempts):
            generator = DataGenerator(input_data[k][:nt])
            print("Analysing trajs={0} n={1} i={2}...".format(j, n, i), end="\r")
            koop = KoopmanModel(n=n, network_lag=lag, verbose=0, nnargs=dict(
                width=width, depth=depth, learning_rate=learning_rate,
                regularization=regularization, dropout=dropout,
                batchnorm=batchnorm[k], lr_factor=5e-3))
            koop.load("models/model-ve-{0}-{1}-{2}.hdf5".format(k, n, i))
            koop.generator = generator
            k_conv[j, i] = koop.estimate_koopman(lag=50)
    np.save(filename, k_conv)